<a href="https://colab.research.google.com/github/YANG-JIEUN/DataStudy-sat/blob/main/231104_%ED%95%9C%EA%B0%95%EB%B0%A9%EB%AC%B8%EA%B0%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
# 코랩 사용 시 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1. 데이터 불러오기&결측치 확인


##1) 기상 데이터 API 가져오기

In [52]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from urllib.parse import quote

In [53]:
# 기상청_지상(종관, ASOS) 일자료 조회서비스 활용을 위한 개인 일반 인증키(Encoding) 저장

key = "fURj5OuFlTtKxCg14GqyGsjoakbzm06eEPKDkzHz8C7fS0vwykajxtBz%2Bv6vrRNg13TTx8CbI1xlRP81fykKeQ%3D%3D"  # 내 인증키

In [54]:
import ssl
from urllib.request import urlopen
context=ssl.create_default_context()
context.set_ciphers("DEFAULT")

In [55]:
# 기상관측일자료목록조회
# url : https://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=fURj5OuFlTtKxCg14GqyGsjoakbzm06eEPKDkzHz8C7fS0vwykajxtBz%2Bv6vrRNg13TTx8CbI1xlRP81fykKeQ%3D%3D&pageNo=1&numOfRows=10&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20220101&endDt=20221231&stnIds=108
# 인증키 지정 : serviceKey = key
# 출력 행수 : numofRows=365 (2022 1년치)
# page 번호 : pageNo = 1


url = 'https://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'+'?serviceKey='+key+'&pageNo=1&numOfRows=365&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20220101&endDt=20221231&stnIds=108'
# stnIds: 지점번호=108이 서울시, api는 서울시 데이터를 옳게 불러왔으므로 이 날씨를 바탕으로 분석을 진행함

# emrList = pd.read_xml(url, xpath='/response/body/items/item')
result = urlopen(url, context=context)
emrList = pd.read_xml(result, xpath='.//item')

emrList.head()

,stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRn,mi10MaxRnHrmt,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
0,108,서울,2022-01-01,-4.3,-10.2,710.0,2.3,1544,NaN,NaN,...,2.7,6.6,10.1,15.1,17.2,1.3,1.8,0.3,None,NaN
1,108,서울,2022-01-02,-1.3,-5.2,2356.0,3.0,1551,NaN,NaN,...,2.7,6.4,10.0,15.0,17.2,1.1,1.6,NaN,{눈}0305-{눈}{강도0}0600-0715. {박무}0610-0705. {박무...,NaN
2,108,서울,2022-01-03,-1.9,-8.0,714.0,2.5,1542,NaN,NaN,...,2.5,6.2,9.8,14.9,17.2,1.1,1.6,0.0,{눈}1815-2055. {눈}2240-{눈}{강도0}2400-,NaN
3,108,서울,2022-01-04,-2.5,-5.6,2400.0,1.0,1445,NaN,NaN,...,2.5,6.1,9.7,14.8,17.1,1.4,2.0,NaN,-{눈}-0055.,NaN
4,108,서울,2022-01-05,-2.8,-7.8,634.0,1.9,1518,NaN,NaN,...,2.4,5.9,9.5,14.7,17.1,1.0,1.4,NaN,{햇무리}0945-1535.,NaN


In [56]:
# 날짜를 datetime으로 바꿔줌
emrList['tm'] = pd.to_datetime(emrList['tm'])
emrList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 62 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   stnId           365 non-null    int64         
 1   stnNm           365 non-null    object        
 2   tm              365 non-null    datetime64[ns]
 3   avgTa           365 non-null    float64       
 4   minTa           364 non-null    float64       
 5   minTaHrmt       364 non-null    float64       
 6   maxTa           365 non-null    float64       
 7   maxTaHrmt       365 non-null    int64         
 8   mi10MaxRn       94 non-null     float64       
 9   mi10MaxRnHrmt   61 non-null     float64       
 10  hr1MaxRn        94 non-null     float64       
 11  hr1MaxRnHrmt    63 non-null     float64       
 12  sumRnDur        147 non-null    float64       
 13  sumRn           147 non-null    float64       
 14  maxInsWs        365 non-null    float64       
 15  maxIns

In [57]:
# case1: 온도별 한강공원 방문객
## ermList 중 [tm(시간), avgTa(평균 기온), minTa(최저 기온), maxTa(최고 기온)] 컬럼 활용
## case1에 저장하기

case1 = emrList[['tm', 'avgTa', 'minTa', 'maxTa']].copy()
case1.tail()

,tm,avgTa,minTa,maxTa
360,2022-12-27,-2.6,-7.3,3.3
361,2022-12-28,-3.3,-6.0,0.1
362,2022-12-29,-2.9,-7.8,2.1
363,2022-12-30,-1.8,-4.4,2.3
364,2022-12-31,-1.2,-5.1,2.1


In [58]:
# case2: 시간과 온도에 따른 한강공원 방문객
## ermList 중 [tm(시간), avgTa(평균 기온), minTa(최저 기온), minTaHrmt(최저 기온 시각), maxTa(최고 기온), maxTaHrmt(최고 기온 시각)] 컬럼 활용
## case2에 저장하기

case2 = emrList[['tm', 'avgTa', 'minTa', 'minTaHrmt', 'maxTa', 'maxTaHrmt']].copy()
case2.head()

,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt
0,2022-01-01,-4.3,-10.2,710.0,2.3,1544
1,2022-01-02,-1.3,-5.2,2356.0,3.0,1551
2,2022-01-03,-1.9,-8.0,714.0,2.5,1542
3,2022-01-04,-2.5,-5.6,2400.0,1.0,1445
4,2022-01-05,-2.8,-7.8,634.0,1.9,1518


In [59]:
# case3: 강수량별 한강공원 방문객
## ermList 중 [tm(시간), avgTa(평균 기온), minTa(최저 기온), maxTa(최고 기온), sumRnDur(강수 계속시간), mi10MaxRn(10분 최다 강수량), mi10MaxRnHrmt(10분 최다 강수량 시각), hr1MaxRn(1시간 최다 강수량), hr1MaxRnHrmt(1시간 최다 강수량 시각), sumRn(일강수량)] 컬럼 활용
## case1에 저장하기

case3 = emrList[['tm', 'avgTa', 'minTa', 'maxTa', 'sumRnDur', 'mi10MaxRn', 'mi10MaxRnHrmt', 'hr1MaxRn', 'hr1MaxRnHrmt', 'sumRn']].copy()
case3.head()

,tm,avgTa,minTa,maxTa,sumRnDur,mi10MaxRn,mi10MaxRnHrmt,hr1MaxRn,hr1MaxRnHrmt,sumRn
0,2022-01-01,-4.3,-10.2,2.3,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,-1.3,-5.2,3.0,4.17,NaN,NaN,NaN,NaN,0.3
2,2022-01-03,-1.9,-8.0,2.5,4.00,NaN,NaN,NaN,NaN,0.0
3,2022-01-04,-2.5,-5.6,1.0,0.92,NaN,NaN,NaN,NaN,0.0
4,2022-01-05,-2.8,-7.8,1.9,NaN,NaN,NaN,NaN,NaN,NaN


##2) 한강공원 주차장 이용현황

In [60]:
# 한강공원 주차장 이용현황 데이터 불러오기
parking_lot = pd.read_csv('/content/drive/My Drive/스터디/오프라인 3회차 (23.11.4)/한강공원 주차장 일별 이용 현황.csv', encoding='euc-kr')
parking_lot.head()

,지구별,주차장명,주차대수,이용시간,날짜
0,PLT-011,잠실1주차장,2497,76141,2023/11/02
1,PLT-002,"난지1,2,3주차장",1739,114787,2023/11/02
2,PLT-007,양화2주차장,25,16,2023/11/02
3,PLT-008,여의도5주차장,340,101503,2023/11/02
4,PLT-009,"반포2,3주차장",3212,157898,2023/11/02


In [61]:
# 2022 1년치만 확인할 것이므로 날짜를 2022년으로 좁혀보자
parking_lot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34887 entries, 0 to 34886
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지구별     34887 non-null  object
 1   주차장명    34887 non-null  object
 2   주차대수    34887 non-null  int64 
 3   이용시간    34887 non-null  int64 
 4   날짜      34887 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.3+ MB


In [ ]:
case1.info()
# 주차장 데이터와 동일하게 날짜가 object 형식임...나중에 datetime으로 바꿔줄 필요가 있어보인다

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tm      365 non-null    object 
 1   avgTa   365 non-null    float64
 2   minTa   364 non-null    float64
 3   maxTa   365 non-null    float64
dtypes: float64(3), object(1)
memory usage: 11.5+ KB


In [62]:
# 날짜를 datetime으로 바꿔주고 2022년 데이터만 확인
parking_lot['날짜'] = pd.to_datetime(parking_lot['날짜'])

In [70]:
parking_lot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34887 entries, 0 to 34886
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   지구별     34887 non-null  object        
 1   주차장명    34887 non-null  object        
 2   주차대수    34887 non-null  int64         
 3   이용시간    34887 non-null  int64         
 4   날짜      34887 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 1.3+ MB


In [82]:
# 2022년 데이터만 추출&역순으로 출력, 인덱스 초기화
parking_lot_2022 = parking_lot[ parking_lot['날짜'].isin(pd.date_range('2022-01-01', '2022-12-31'))].loc[::-1].reset_index(drop=True)
parking_lot_2022

,지구별,주차장명,주차대수,이용시간,날짜
0,PLT-003,망원1주차장,317,19329,2022-01-01
1,PLT-009,반포1주차장,901,36061,2022-01-01
2,PLT-003,"망원2,3주차장",1544,107166,2022-01-01
3,PLT-007,양화1주차장,257,15670,2022-01-01
4,PLT-008,여의도3주차장,535,38874,2022-01-01
...,...,...,...,...,...
9106,PLT-008,여의도5주차장,129,19176,2022-12-31
9107,PLT-008,여의도2주차장,655,54893,2022-12-31
9108,PLT-010,잠원2-6주차장,2328,216879,2022-12-31
9109,PLT-001,강서1주차장,138,7007,2022-12-31


- 1주차장, 2주차장으로 표기되어있지만 같은 한강공원을 지칭하는 것
- 지구별로 방문객을 합산하는 것이 분석 목적과 더 부합하다고 판단

In [86]:
# 지구별로 분류, 똑같은 날짜끼리는 주차대수를 합산하기로 함
parking_lot_2022['전체주차대수'] = parking_lot_2022.groupby(['지구별', '날짜'])['주차대수'].sum()
parking_lot_2022

TypeError: ignored

##3) 공공자전거(따릉이) 대여 현황

In [ ]:
# 따릉이 대여건수 1~6월
rental1 = pd.read_csv('/content/drive/My Drive/스터디/오프라인 3회차 (23.11.4)/서울특별시 공공자전거 일별 대여건수_22.01-22.06.csv', encoding='euc-kr')
rental1.head()

,대여일시,대여건수,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,2022-01-01,"29,185",NaN,NaN,NaN,NaN
1,2022-01-02,"28,914",NaN,NaN,NaN,NaN
2,2022-01-03,"53,573",NaN,NaN,NaN,NaN
3,2022-01-04,"56,344",NaN,NaN,NaN,NaN
4,2022-01-05,"57,413",NaN,NaN,NaN,NaN


In [ ]:
# 따릉이 대여건수 7~12월
rental2 = pd.read_csv('/content/drive/My Drive/스터디/오프라인 3회차 (23.11.4)/서울특별시 공공자전거 일별 대여건수_22.07-22.12.csv', encoding='euc-kr')
rental2.head()

,대여일시,대여건수,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,2022-07-01,"157,478",NaN,NaN,NaN,NaN
1,2022-07-02,"131,815",NaN,NaN,NaN,NaN
2,2022-07-03,"114,470",NaN,NaN,NaN,NaN
3,2022-07-04,"151,497",NaN,NaN,NaN,NaN
4,2022-07-05,"149,259",NaN,NaN,NaN,NaN


In [ ]:
# 형태 확인
print(rental1.shape, rental2.shape)

(213, 6) (210, 6)


In [ ]:
# 행을 합쳤을 때 365가 넘게 나옴....중복값 확인
rental1[rental1.duplicated()]

,대여일시,대여건수,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
182,NaN,NaN,NaN,NaN,NaN,NaN
183,NaN,NaN,NaN,NaN,NaN,NaN
184,NaN,NaN,NaN,NaN,NaN,NaN
185,NaN,NaN,NaN,NaN,NaN,NaN
186,NaN,NaN,NaN,NaN,NaN,NaN
187,NaN,NaN,NaN,NaN,NaN,NaN
188,NaN,NaN,NaN,NaN,NaN,NaN
189,NaN,NaN,NaN,NaN,NaN,NaN
190,NaN,NaN,NaN,NaN,NaN,NaN
191,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 결측치 확인
rental1.isnull().sum()

대여일시           32
대여건수           32
Unnamed: 2    213
Unnamed: 3    213
Unnamed: 4    213
Unnamed: 5    213
dtype: int64

In [ ]:
# 결측치 행 제거
rental1 = rental1[['대여일시', '대여건수']]
rental2 = rental2[['대여일시', '대여건수']]

In [ ]:
# 결측치 제거
rental1 = rental1.dropna(axis=0)
rental2 = rental2.dropna(axis=0)

In [ ]:
# 해치웠나?
print(rental1.isnull().sum(), rental2.isnull().sum())

대여일시    0
대여건수    0
dtype: int64 대여일시    0
대여건수    0
dtype: int64


In [ ]:
# 결측치 제거 후 shape 확인
print(rental1.shape, rental2.shape)  # 365로 1년치 데이터가 완성됐다!

(181, 2) (184, 2)


In [ ]:
# 이제 2022 1년치 데이터로 만들어보자
# 위/아래로 합치기 - 행 기준
rental=pd.concat([rental1, rental2], axis = 0)

# rental: 서울시 공공자전거 대여 현황 2022 전체
rental.shape

(365, 2)

데이터 확인 결과, 구체적인 대여 장소 및 위치 명시가 없어 위 두 데이터와 연결하기 어렵고 정확한 분석이 힘들다고 판단

**→해당 데이터 관련 분석은 진행하지 않음.**

#2. 시각화

##1) 기온별 한강공원 방문자수

# 요약
1. 날씨에 따른 한강 방문객을 확인하고자 공공자전거 대여 데이터와 한강공원 주차장 데이터를 구했으나, 공공자전거 대여 데이터의 경우 대여날짜와 건수만 나와있어 적합하지 않다고 판단, **주차장 데이터로만 분석을 진행함.**
2.

# 추가로 진행할 분석
1. 주차대수가 많은 한강공원의 경우, 한강공원 조성이 다른 곳들과 어떻게 다른가?
2. 주차대수가 많은 날짜의 경우 근처에서 어떤 행사가 진행되었나?